## Install and Load libraries

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.1 MB/s 
     |████████████████████████████████| 4.4 MB 85.4 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 212 kB 81.3 MB/s 
     |████████████████████████████████| 140 kB 94.9 MB/s 
     |████████████████████████████████| 1.1 MB 68.1 MB/s 
     |████████████████████████████████| 596 kB 70.1 MB/s 
     |████████████████████████████████| 127 kB 96.2 MB/s 
     |████████████████████████████████| 6.6 MB 64.6 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 271 kB 93.9 MB/s 
     |████████████████████████████████| 144 kB 75.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing

In [2]:
from datasets import load_dataset, load_metric
import requests
import json
import os
import torch
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

if not os.path.exists('/content/drive/MyDrive/BERT-SQuAD'):
  os.mkdir('/content/drive/MyDrive/BERT-SQuAD')

Mounted at /content/drive


In [4]:
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2022-07-26 17:22:11--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   167MB/s    in 0.2s    

2022-07-26 17:22:14 (167 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2022-07-26 17:22:15--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [5]:
import transformers

print(transformers.__version__)

4.20.1


In [41]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = True
#model_checkpoint = "distilbert-base-cased"
model_checkpoint = "SpanBERT/spanbert-base-cased"
#model_checkpoint = 'drive/MyDrive/w266/model_checkpoints/SpanBert/SpanBert-NQ-cased-trained'
batch_size = 16

In [7]:
# with open('train-v2.0.json', 'rb') as f:
#   train_data = json.load(f)
# with open('dev-v2.0.json', 'rb') as f:
#   test_data = json.load(f)
# #data dict keys are title and paragraph
# # print(train_data['data'])
# # print(test_data['data'])

# train_data = train_data['data']


# train_path = 'train-v2.0.json'
# train_data = load_dataset('json', data_files=train_path)
# # data = load_dataset('json', data_files=train_path)

Using custom data configuration default-28dc4ff952d745ac


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-28dc4ff952d745ac/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [86]:
from datasets import load_dataset
dataset_train_val = load_dataset('squad_v2')

  0%|          | 0/2 [00:00<?, ?it/s]

In [84]:
dataset_train_test = load_dataset('squad_v2')
dataset_train_test = dataset_train_test['train']
dataset_train_test = dataset_train_test.train_test_split(test_size=0.1)
dataset_train_test
#dataset_temp['train'], dataset_temp['test'] = dataset_temp.train_test_split(.1).values()
# dataset_temp = dataset_temp['train']
# dataset_temp = dataset_temp.train_test_split(test_size=0.1)

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 117287
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 13032
    })
})

In [87]:
dataset_train_val

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [28]:
# import numpy as np
# randnums= np.random.randint(1,130319,11873)
# test_data = []
# for i in range(len(randnums)):
#   train_num = int(randnums[i])
#   test_data.append(dataset['train'][train_num])

In [35]:
# test_data[:10]

[{'answers': {'answer_start': [12], 'text': ['democratic']},
  'context': "Greece is a democratic and developed country with an advanced high-income economy, a high quality of life and a very high standard of living. A founding member of the United Nations, Greece was the tenth member to join the European Communities (precursor to the European Union) and has been part of the Eurozone since 2001. It is also a member of numerous other international institutions, including the Council of Europe, NATO,[a] OECD, OIF, OSCE and the WTO. Greece, which is one of the world's largest shipping powers, middle powers and top tourist destinations, has the largest economy in the Balkans, where it is an important regional investor.",
  'id': '572618f238643c19005acfec',
  'question': 'Which political philosophy does Greece follow?',
  'title': 'Greece'},
 {'answers': {'answer_start': [410],
   'text': ['the World Federation of Exchanges']},
  'context': "There are four hundred securities listed on the s

In [20]:
#datasets.save_to_disk('drive/MyDrive/w266/data/natural_questions/train_test_master.json')

## Preprocessing the training data

We will use TokenizerFast and leverage a few properties for QA task, such as return offset mapping etc. 

In [42]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/413 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

In [43]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [44]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 1184, 1110, 1240, 1271, 136, 102, 1139, 1271, 1110, 188, 7777, 2497, 1394, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [45]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [46]:
pad_on_right = tokenizer.padding_side == "right"

In [26]:
#train_data = train_data['data']

 Tokenize with truncation and padding, but keep the overflows using a stride. This results in one example possible giving several features when a context is long, each of those features having a context that overlaps a bit the context of the previous feature.

In [27]:
def test_prepare_train_features(examples):
    # remove white space in question section 
    # left whitespace

    # for sentence in squad['data']:
    #   for passage in sentence['paragraphs']:
    #     context = passage['context']
    #     for question_answer in passage['qas']:
    #       question = question_answer['question']
    #       for answer in question_answer['answers']:
    #         contexts.append(context)
    #         questions.append(question)
    #         answers.append(answer)

    # for i in range(len(examples['data'])):
    #   example = examples['data'][i]
    #   for j in range(len(example)):
    #     paragraph = example[j]['paragraphs']
    #     for k in range(len(paragraph)):
    #       context = paragraph[k]['context']
    #       qas = paragraph[k]['qas']
    #       for l in range(len(qas)):
    #         question = qas[l]['question']
    #         answers = qas[l]['answers']
    
    # question = []
    # for i in range(len(examples['data'])):
    #   paragraphs = examples['data'][1]
      #question.append(examples['data'][1]['paragraphs'][0]['qas'][0]['question'])

        #print(example, paragraph)
      # for j in range(len(sentence['paragraphs'])):
      #   context = sentence['paragraphs'][i]["context"]
      # for passage in sentence['paragraphs']:
      #   context = passage['context']
        # for question_answer in passage['qas']:
        #   question = [q.lstrip() for q in question_answer['question']]
  
    # Tokenize with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    # tokenized_examples = tokenizer(
    #     question if pad_on_right else context,
    #     context if pad_on_right else question,
    #     truncation="only_second" if pad_on_right else "only_first",
    #     max_length=max_length,
    #     stride=doc_stride,
    #     return_overflowing_tokens=True,
    #     return_offsets_mapping=True,
    #     padding="max_length",
    # )

    # maps feature back to sample
    # sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # # map from token to character position in the original context. 
    # offset_mapping = tokenized_examples.pop("offset_mapping")

    # # # Generate labels
    # tokenized_examples["start_positions"] = []
    # tokenized_examples["end_positions"] = []

    # for i, offsets in enumerate(offset_mapping):
    #     # label impossible answers with the index of the CLS token.
    #     input_ids = tokenized_examples["input_ids"][i]
    #     cls_index = input_ids.index(tokenizer.cls_token_id)

    #     # map back to the original question/context 
    #     sequence_ids = tokenized_examples.sequence_ids(i)

    #     # One example can give several spans, this is the index of the example containing this span of text.
    #     sample_index = sample_mapping[i]
    #     answers = qas[l]['answers'][sample_index]
        # If no answers are given, set the cls_index as answer.

        
        # if len(answers["answer_start"]) == 0:
        #     tokenized_examples["start_positions"].append(cls_index)
        #     tokenized_examples["end_positions"].append(cls_index)
        # else:
        #     # Start/end character index of the answer in the text.
        #     start_char = answers["answer_start"][0]
        #     end_char = start_char + len(answers["text"][0])

    #         # Start token index of the current span in the text.
    #         token_start_index = 0
    #         while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
    #             token_start_index += 1

    #         # End token index of the current span in the text.
    #         token_end_index = len(input_ids) - 1
    #         while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
    #             token_end_index -= 1

    #         # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
    #         if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    #             tokenized_examples["start_positions"].append(cls_index)
    #             tokenized_examples["end_positions"].append(cls_index)
    #         else:
    #             # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
    #             # Note: we could go after the last offset if the answer is the last word (edge case).
    #             while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
    #                 token_start_index += 1
    #             tokenized_examples["start_positions"].append(token_start_index - 1)
    #             while offsets[token_end_index][1] >= end_char:
    #                 token_end_index -= 1
    #             tokenized_examples["end_positions"].append(token_end_index + 1)

    # return tokenized_examples
    return examples

In [50]:
def prepare_train_features(examples):
    # remove white space in question section 
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # maps feature back to sample
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # map from token to character position in the original context. 
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Generate labels
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # map back to the original question/context 
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [88]:
tokenized_datasets = dataset_train_val.map(prepare_train_features, batched=True, remove_columns=dataset_train_val["train"].column_names)

In [89]:
tokenized_datasets['train'].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'end_positions': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'start_positions': Value(dtype='int64', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [90]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 132932
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 12298
    })
})

In [31]:
#tokenized_train = train_data.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

In [35]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Fine-tuning the model

In [36]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on

In [39]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-SQuAD",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01
)

    # push_to_hub=True,

In [40]:
from transformers import default_data_collator

data_collator = default_data_collator

In [41]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


    # eval_dataset=tokenized_datasets["validation"],

In [ ]:
trainer.predict()

In [42]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 132079
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8255


Epoch,Training Loss,Validation Loss
1,1.297000,1.319013


Saving model checkpoint to distilbert-base-cased-finetuned-SQuAD/checkpoint-500
Configuration saved in distilbert-base-cased-finetuned-SQuAD/checkpoint-500/config.json
Model weights saved in distilbert-base-cased-finetuned-SQuAD/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-base-cased-finetuned-SQuAD/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-base-cased-finetuned-SQuAD/checkpoint-500/special_tokens_map.json
Saving model checkpoint to distilbert-base-cased-finetuned-SQuAD/checkpoint-1000
Configuration saved in distilbert-base-cased-finetuned-SQuAD/checkpoint-1000/config.json
Model weights saved in distilbert-base-cased-finetuned-SQuAD/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in distilbert-base-cased-finetuned-SQuAD/checkpoint-1000/tokenizer_config.json
Special tokens file saved in distilbert-base-cased-finetuned-SQuAD/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to distilbert-base-case

TrainOutput(global_step=8255, training_loss=1.6006293793001873, metrics={'train_runtime': 2779.0259, 'train_samples_per_second': 47.527, 'train_steps_per_second': 2.97, 'total_flos': 1.2942398099344896e+16, 'train_loss': 1.6006293793001873, 'epoch': 1.0})

In [43]:
trainer.save_model("drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-SQuAD-cased-trained")

Saving model checkpoint to drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-SQuAD-cased-trained
Configuration saved in drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-SQuAD-cased-trained/config.json
Model weights saved in drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-SQuAD-cased-trained/pytorch_model.bin
tokenizer config file saved in drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-SQuAD-cased-trained/tokenizer_config.json
Special tokens file saved in drive/MyDrive/w266/model_checkpoints/DistilBert/distilBert-SQuAD-cased-trained/special_tokens_map.json


In [44]:
!cp -R distilbert-base-cased-finetuned-SQuAD drive/MyDrive/w266/model_checkpoints/DistilBert/

In [ ]:
# !cp -R test-NQ-trained drive/MyDrive/w266/model_checkpoints/DistilBert/

In [ ]:
# trainer.save_model("drive/MyDrive/w266/model_checkpoints/DistilBert/test-NQ-trained/")

In [ ]:
# !ls drive/MyDrive/w266/model_checkpoints/DistilBert/test-NQ-trained

In [ ]:
!ls drive/MyDrive/w266/model_checkpoints/DistilBert/

distilbert-base-cased-finetuned-NQ    distilBert-NQ-cased-trained
distilbert-base-uncased-finetuned-NQ


In [45]:
model.save_pretrained("DistilBERT-cased-SQuAD")

Configuration saved in DistilBERT-cased-SQuAD/config.json
Model weights saved in DistilBERT-cased-SQuAD/pytorch_model.bin


In [46]:
!cp -R DistilBERT-cased-SQuAD drive/MyDrive/w266/model_checkpoints/DistilBert/

In [47]:
!ls drive/MyDrive/w266/model_checkpoints/DistilBert/DistilBERT-cased-SQuAD

config.json  pytorch_model.bin


In [48]:
from transformers import DistilBertForQuestionAnswering

bert = DistilBertForQuestionAnswering.from_pretrained("/content/drive/MyDrive/w266/model_checkpoints/DistilBert/DistilBERT-cased-SQuAD/")

loading configuration file /content/drive/MyDrive/w266/model_checkpoints/DistilBert/DistilBERT-cased-SQuAD/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "vocab_size": 28996
}

loading weights file /content/drive/MyDrive/w266/model_checkpoints/DistilBert/DistilBERT-cased-SQuAD/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were init

## Evaluation

In [49]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [50]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [51]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([ 49,  37,  73,  81, 164,   0,  44,   0,   0, 205, 122,  42,  22,  31,
           0, 122], device='cuda:0'),
 tensor([ 49,  40,  77,  82, 164,  19,  44,   0, 161, 209, 123,  44,  26,  33,
           0, 123], device='cuda:0'))

Post processing : 
- the start position could be greater than the end position, or point to a span of text in the question instead of the answer. In that case, we might want to look at the second best prediction to see if it gives a possible answer and select that instead.

By:

- score: adding the start and end logits
- hyper-parameter we call n_best_size

In [52]:
n_best_size = 20

In [53]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # back the original substring corresponding to the answer in the context
                }
            )

### process validation set:
- will need example id and offset mapping needed

In [47]:
def prepare_validation_features(examples):

    # similar process as train
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # need example_id for matching
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # mao sequence corresponding to that example 
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

Map the validation set

In [91]:
dataset_train_test = prepare_validation_features(dataset_train_test)

KeyError: ignored

Get predictions for all features by using the `Trainer.predict` method:

In [49]:
raw_predictions = trainer.predict(validation_features)

NameError: ignored

The `Trainer` *hides* the columns that are not used by the model (here `example_id` and `offset_mapping` which we will need for our post-processing).

In [58]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

Upper limit to avoid very long answers from our considerations.

In [61]:
max_answer_length = 30

In [62]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 14.478552, 'text': 'France'},
 {'score': 9.711885, 'text': 'France.'},
 {'score': 8.393439, 'text': 'a region in France'},
 {'score': 7.3407574, 'text': 'region in France'},
 {'score': 6.9690003, 'text': 'Normandy, a region in France'},
 {'score': 6.9054623, 'text': 'in France'},
 {'score': 4.6485715,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway'},
 {'score': 4.1371818, 'text': ', a region in France'},
 {'score': 4.094509,
  'text': 'French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': 4.005614, 'text': 'France. They were descended from Norse'},
 {'score': 3.6539826,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark'},
 {'score': 3.6267734, 'text': 'a region in France.'},
 {'score': 3.4058015,
  'text': 'France. They were desce

Compare to ground-truth answer:

In [63]:
dataset["validation"][0]["answers"]

{'answer_start': [159, 159, 159, 159],
 'text': ['France', 'France', 'France', 'France']}

Map feature back to sample through example id: 

In [64]:
import collections

examples = dataset["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

## Post-processing function
- predict the impossible answer when that score is greater than the score of the best non-impossible answer

In [65]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # collect results
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    #  all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # map some the positions in our logits to span of texts in the original context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

And we can apply our post-processing function to our raw predictions:

In [66]:
final_predictions = postprocess_qa_predictions(dataset["validation"], validation_features, raw_predictions.predictions)

Post-processing 11873 example predictions split into 12199 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

Use the squad v2 metric for evluation

In [67]:
metric = load_metric("squad_v2")

## Final Evaluation

In [68]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 57.47300944669366,
 'HasAns_f1': 63.4828568649514,
 'HasAns_total': 5928,
 'NoAns_exact': 63.63330529857023,
 'NoAns_f1': 63.63330529857023,
 'NoAns_total': 5945,
 'best_exact': 60.77655184030995,
 'best_exact_thresh': 0.0,
 'best_f1': 63.66089614586712,
 'best_f1_thresh': 0.0,
 'exact': 60.557567590331004,
 'f1': 63.55818878930633,
 'total': 11873}